In [1]:
import pandas as pd
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.svm import LinearSVC
from sklearn.calibration import CalibratedClassifierCV
from sklearn.metrics import precision_recall_fscore_support, classification_report, auc, roc_curve
from scipy.sparse import hstack
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from datasets import load_dataset, Image, concatenate_datasets
from torch.utils.data import DataLoader, Dataset, TensorDataset
from utils import *

In [ ]:
import warnings

def fxn():
    warnings.warn("deprecated", DeprecationWarning)

with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    fxn()

In [ ]:
# importing the data
datafolder = '../../data/'
train = datafolder+'train_with_features.csv'
test = datafolder+'test_with_features.csv'
dev = datafolder+'dev_with_features.csv'
test_unseen = datafolder+'test_unseen_with_features.csv'
dev_unseen = datafolder+'dev_unseen_with_features.csv'
df_train_org = pd.read_csv(train, skip_blank_lines=False)
df_dev = pd.read_csv(dev, skip_blank_lines=False)
df_dev_unseen = pd.read_csv(dev_unseen, skip_blank_lines=False)
df_test = pd.read_csv(test, skip_blank_lines=False)
df_test_unseen = pd.read_csv(test_unseen, skip_blank_lines=False)
df_train = pd.concat([df_dev, df_test], ignore_index=True, axis=0)
df_test = df_test_unseen
df_dev = df_dev_unseen
df_train = df_train.fillna('')

# ResNet50:

In [ ]:
train_img_org = torch.stack(torch.load('image_embeddings/train_img_tensors.pt'))
train_img1 = torch.stack(torch.load('image_embeddings/dev_img_tensors.pt'))
train_img2 = torch.stack(torch.load('image_embeddings/test_img_tensors.pt'))

dev_img = torch.stack(torch.load('image_embeddings/dev_unseen_img_tensors.pt'))
test_img = torch.stack(torch.load('image_embeddings/test_unseen_img_tensors.pt'))

train_img = torch.cat((train_img1,train_img2), dim=0)

Y_train_org = df_train_org['label']
Y_train = df_train['label']
Y_dev = df_dev['label']
Y_test = df_test['label']

train_X_org = [np.array(x.cpu()).flatten() for x in train_img_org]
train_X = [np.array(x.cpu()).flatten() for x in train_img]
dev_X = [np.array(x.cpu()).flatten() for x in dev_img]
test_X = [np.array(x.cpu()).flatten() for x in test_img]

In [5]:
# from sklearn.svm import SVC
# clf_svc = SVC(kernel='linear', C=10, probability = True) 
# clf_svc.fit(train_X_org, Y_train_org)

SVC(C=10, kernel='linear', probability=True)

### Note: This model took approx 18 hours to run. Hence it is better if saved to disk: size on disk 4.3 GB

In [8]:
import pickle

# # Save model to disk
# with open('models/ResNet50_lin_c10_model.pkl', 'wb') as f:
#     pickle.dump(clf_svc, f)


# This will save the `clf_svc` object to a file named `ResNet50_lin_c10_model.pkl` in binary mode (`'wb'`). To load the model from disk, you can use the `pickle.load()` function as follows:


# Load saved model
with open('models/ResNet50_lin_c10_model.pkl', 'rb') as f:
    clf_svc = pickle.load(f)



In [9]:
y_dev_proba_ResNet = clf_svc.predict_proba(dev_X)
y_test_proba_ResNet = clf_svc.predict_proba(test_X)
y_train_proba_ResNet = clf_svc.predict_proba(train_X)

# BoW:

In [13]:
train_original = datafolder+'train_with_features.csv'
df_train_original = pd.read_csv(train_original, skip_blank_lines=False)
Y_train_original = df_train_original['label']
df_train_original = df_train_original.fillna('')

In [14]:
vectorizer_BoW = CountVectorizer(tokenizer=lambda x: x.split(), analyzer='word', ngram_range=(1, 1)) # to build 1n-grams from the word ==> BoW
                
X_train_original_BoW = vectorizer_BoW.fit_transform(df_train_original.tokens)
X_train_BoW = vectorizer_BoW.transform(df_train.tokens)
X_dev_BoW = vectorizer_BoW.transform(df_dev.tokens) 
X_test_BoW = vectorizer_BoW.transform(df_test.tokens)

clf = LinearSVC(max_iter=1000000, C = 10,random_state =456) 
clf_svc_BoW = CalibratedClassifierCV(clf) 

clf_svc_BoW.fit(X_train_original_BoW, Y_train_original)

y_train_proba_BoW = clf_svc_BoW.predict_proba(X_train_BoW)
y_dev_proba_BoW = clf_svc_BoW.predict_proba(X_dev_BoW)
y_test_proba_BoW = clf_svc_BoW.predict_proba(X_test_BoW)


In [25]:
from utils import *

In [26]:
X_train_Bow = fuse_proba(y_train_proba_BoW, y_train_proba_ResNet)
X_dev_Bow = fuse_proba(y_dev_proba_BoW, y_dev_proba_ResNet)
X_test_Bow = fuse_proba(y_test_proba_BoW, y_test_proba_ResNet)

BoW_ResNet_results = late_fuse_MLPX(X_train_Bow, Y_train, X_dev_Bow, Y_dev, X_test_Bow, Y_test)

In [24]:
BoW_ResNet_results

(   f1_score  precision    recall  accuracy     AUROC
 0  0.514628    0.57903  0.571912  0.516667  0.571912,
    f1_score  precision    recall  accuracy     AUROC
 0  0.517674   0.561345  0.559467     0.518  0.559467)

In [27]:
late_fuse_MLPX(X_train_Bow, Y_train, X_dev_Bow, Y_dev, X_test_Bow, Y_test)

(   f1_score  precision  recall  accuracy  AUROC
 0  0.527063    0.55305   0.555  0.527778  0.555,
    f1_score  precision    recall  accuracy     AUROC
 0  0.551913   0.565965  0.569867     0.555  0.569867)

In [28]:
late_fuse_MLPX(X_train_Bow, Y_train, X_dev_Bow, Y_dev, X_test_Bow, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.505513   0.580559  0.570147  0.509259  0.570147,
    f1_score  precision    recall  accuracy     AUROC
 0  0.506816   0.558596  0.554933     0.508  0.554933)

In [29]:
late_fuse_MLPX(X_train_Bow, Y_train, X_dev_Bow, Y_dev, X_test_Bow, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.535577   0.536269  0.537941  0.555556  0.537941,
    f1_score  precision    recall  accuracy     AUROC
 0  0.564338   0.564001  0.564933    0.5885  0.564933)

In [30]:
late_fuse_MLPX(X_train_Bow, Y_train, X_dev_Bow, Y_dev, X_test_Bow, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.505533   0.508818  0.507941  0.559259  0.507941,
    f1_score  precision  recall  accuracy   AUROC
 0  0.554545     0.5677  0.5568     0.611  0.5568)

In [31]:
late_fuse_MLPX(X_train_Bow, Y_train, X_dev_Bow, Y_dev, X_test_Bow, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.526684   0.529004  0.530735  0.542593  0.530735,
    f1_score  precision    recall  accuracy     AUROC
 0   0.56961   0.569356  0.571867    0.5885  0.571867)

In [32]:
late_fuse_MLPX(X_train_Bow, Y_train, X_dev_Bow, Y_dev, X_test_Bow, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.519644    0.57227  0.568676   0.52037  0.568676,
    f1_score  precision    recall  accuracy     AUROC
 0   0.52797   0.560566  0.561067     0.528  0.561067)

In [33]:
late_fuse_MLPX(X_train_Bow, Y_train, X_dev_Bow, Y_dev, X_test_Bow, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.527559   0.528932  0.527647  0.568519  0.527647,
    f1_score  precision    recall  accuracy     AUROC
 0  0.555791   0.562918  0.556267     0.603  0.556267)

In [34]:
late_fuse_MLPX(X_train_Bow, Y_train, X_dev_Bow, Y_dev, X_test_Bow, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.530305   0.537453  0.540147  0.538889  0.540147,
    f1_score  precision    recall  accuracy     AUROC
 0   0.56474   0.565826  0.569333     0.579  0.569333)

In [35]:
late_fuse_MLPX(X_train_Bow, Y_train, X_dev_Bow, Y_dev, X_test_Bow, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.513105   0.574405  0.568382  0.514815  0.568382,
    f1_score  precision    recall  accuracy     AUROC
 0   0.52249   0.558821  0.558533    0.5225  0.558533)

In [80]:
run_multiple_times(X_train_Bow, Y_train, X_dev_Bow, Y_dev, X_test_Bow, Y_test)

Average Development: 
   f1_score  precision    recall  accuracy     AUROC
0  0.515675   0.541404  0.539809  0.540926  0.539809 
Average Test: 
   f1_score  precision    recall  accuracy     AUROC
0  0.542267   0.564144  0.559493     0.571  0.559493


# Char-n-gram:

In [36]:
vectorizer_char = CountVectorizer(tokenizer=lambda x: list(x), analyzer='char', ngram_range=(1, 3)) # to build 1n-grams from the word ==> BoW

X_train_original_char = vectorizer_char.fit_transform(df_train_original.tokens)
X_train_char = vectorizer_char.transform(df_train.tokens)
X_dev_char = vectorizer_char.transform(df_dev.tokens) 
X_test_char = vectorizer_char.transform(df_test.tokens)

clf = LinearSVC(max_iter=1000000, C = 10, random_state =456) 
clf_svc_char = CalibratedClassifierCV(clf) 
clf_svc_char.fit(X_train_original_char, Y_train_original)

y_train_proba_char = clf_svc_char.predict_proba(X_train_char)
y_dev_proba_char = clf_svc_char.predict_proba(X_dev_char)
y_test_proba_char = clf_svc_char.predict_proba(X_test_char)

C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\svm\_base.py:1225: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


In [37]:
X_train_char = fuse_proba(y_train_proba_char, y_train_proba_ResNet)
X_dev_char = fuse_proba(y_dev_proba_char, y_dev_proba_ResNet)
X_test_char = fuse_proba(y_test_proba_char, y_test_proba_ResNet)

char_ResNet_results = late_fuse_MLPX(X_train_char, Y_train, X_dev_char, Y_dev, X_test_char, Y_test)

In [38]:
char_ResNet_results

(   f1_score  precision    recall  accuracy     AUROC
 0  0.513105   0.533727  0.535441  0.514815  0.535441,
    f1_score  precision    recall  accuracy     AUROC
 0  0.534239   0.549456  0.552267     0.537  0.552267)

In [39]:
late_fuse_MLPX(X_train_char, Y_train, X_dev_char, Y_dev, X_test_char, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.471875   0.486377  0.490588  0.568519  0.490588,
    f1_score  precision    recall  accuracy     AUROC
 0  0.520414   0.555681  0.536267     0.612  0.536267)

In [40]:
late_fuse_MLPX(X_train_char, Y_train, X_dev_char, Y_dev, X_test_char, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.492759   0.497436  0.497794  0.555556  0.497794,
    f1_score  precision    recall  accuracy     AUROC
 0  0.534193   0.549073  0.539467     0.599  0.539467)

In [41]:
late_fuse_MLPX(X_train_char, Y_train, X_dev_char, Y_dev, X_test_char, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.469457   0.490146  0.493824  0.577778  0.493824,
    f1_score  precision    recall  accuracy     AUROC
 0  0.514069   0.558856  0.534933     0.616  0.534933)

In [42]:
late_fuse_MLPX(X_train_char, Y_train, X_dev_char, Y_dev, X_test_char, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0    0.4908   0.497169  0.497647  0.559259  0.497647,
    f1_score  precision    recall  accuracy     AUROC
 0   0.53241   0.549745  0.538933     0.601  0.538933)

In [43]:
late_fuse_MLPX(X_train_char, Y_train, X_dev_char, Y_dev, X_test_char, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.516049   0.516017  0.516176  0.546296  0.516176,
    f1_score  precision    recall  accuracy     AUROC
 0  0.545639   0.546703  0.545333      0.58  0.545333)

In [44]:
late_fuse_MLPX(X_train_char, Y_train, X_dev_char, Y_dev, X_test_char, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.517718    0.54292  0.544559  0.518519  0.544559,
    f1_score  precision  recall  accuracy   AUROC
 0  0.528344   0.549504  0.5516    0.5295  0.5516)

In [45]:
late_fuse_MLPX(X_train_char, Y_train, X_dev_char, Y_dev, X_test_char, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0   0.51208   0.512135  0.512059  0.546296  0.512059,
    f1_score  precision  recall  accuracy   AUROC
 0  0.547008   0.549347  0.5468    0.5855  0.5468)

In [46]:
late_fuse_MLPX(X_train_char, Y_train, X_dev_char, Y_dev, X_test_char, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.471472   0.479219  0.483971  0.553704  0.483971,
    f1_score  precision  recall  accuracy  AUROC
 0  0.533879   0.561519   0.544     0.613  0.544)

In [47]:
late_fuse_MLPX(X_train_char, Y_train, X_dev_char, Y_dev, X_test_char, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.508097    0.50814  0.508088  0.542593  0.508088,
    f1_score  precision    recall  accuracy     AUROC
 0  0.543618    0.54573  0.543467     0.582  0.543467)

In [81]:
run_multiple_times(X_train_char, Y_train, X_dev_char, Y_dev, X_test_char, Y_test)

C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Average Development: 
   f1_score  precision    recall  accuracy     AUROC
0  0.488398   0.496267  0.516074  0.550185  0.516074 
Average Test: 
   f1_score  precision    recall  accuracy     AUROC
0  0.514967   0.527744  0.540493   0.57555  0.540493


# Advanced SVM:

In [48]:
df_train_original = df_train_original.fillna('')
df_train = df_train.fillna('')
df_dev = df_dev.fillna('')
df_test = df_test.fillna('')

In [51]:
vectorizer1 = CountVectorizer(tokenizer=lambda x: x.split(), analyzer='word', ngram_range=(1, 3)) # to build n-grams (n=1-3) from the pos_fw_emo representation
vectorizer2 = CountVectorizer(tokenizer=lambda x: x.split(), analyzer='word', ngram_range=(1, 1)) # unigrams of emotion associations
vectorizer3 = CountVectorizer(tokenizer=lambda x: x.split(), analyzer='word', ngram_range=(1, 1)) # unigrams of tokens (BoW)
vectorizer4 = CountVectorizer(tokenizer=lambda x: x.split(), analyzer='word', ngram_range=(1, 4)) # unigrams of intent (BoW)


# combine the features
X_train_original_advanced = hstack((vectorizer1.fit_transform(df_train_original.pos_fw_emo), vectorizer2.fit_transform(df_train_original.emotion_associations), df_train_original[['count']].values, \
                df_train_original[['sentiment_score']].values, vectorizer3.fit_transform(df_train_original.tokens), vectorizer4.fit_transform(df_train_original.intent)), format='csr') 

X_train_advanced = hstack((vectorizer1.transform(df_train.pos_fw_emo), vectorizer2.transform(df_train.emotion_associations), df_train[['count']].values, \
                df_train[['sentiment_score']].values, vectorizer3.transform(df_train.tokens), vectorizer4.transform(df_train.intent) ), format='csr') 

X_dev_advanced = hstack((vectorizer1.transform(df_dev.pos_fw_emo), vectorizer2.transform(df_dev.emotion_associations), df_dev[['count']].values, \
                df_dev[['sentiment_score']].values, vectorizer3.transform(df_dev.tokens), vectorizer4.transform(df_dev.intent) ), format='csr') 

X_test_advanced = hstack((vectorizer1.transform(df_test.pos_fw_emo), vectorizer2.transform(df_test.emotion_associations), df_test[['count']].values, \
                df_test[['sentiment_score']].values, vectorizer3.transform(df_test.tokens), vectorizer4.transform(df_test.intent) ), format='csr') 

clf = LinearSVC(max_iter=1000000, C = 10, random_state =456) 
clf_svc_advanced = CalibratedClassifierCV(clf) 
clf_svc_advanced.fit(X_train_original_advanced, Y_train_original)

y_train_proba_advanced = clf_svc_advanced.predict_proba(X_train_advanced)
y_dev_proba_advanced = clf_svc_advanced.predict_proba(X_dev_advanced)
y_test_proba_advanced = clf_svc_advanced.predict_proba(X_test_advanced)

In [52]:
X_train_advanced = fuse_proba(y_train_proba_advanced, y_train_proba_ResNet)
X_dev_advanced = fuse_proba(y_dev_proba_advanced, y_dev_proba_ResNet)
X_test_advanced = fuse_proba(y_test_proba_advanced, y_test_proba_ResNet)

advanced_ResNet_results = late_fuse_MLPX(X_train_advanced, Y_train, X_dev_advanced, Y_dev, X_test_advanced, Y_test)

In [53]:
advanced_ResNet_results

(   f1_score  precision    recall  accuracy     AUROC
 0  0.481243   0.566638  0.525294  0.627778  0.525294,
    f1_score  precision    recall  accuracy     AUROC
 0  0.484958   0.577775  0.529333     0.628  0.529333)

In [54]:
late_fuse_MLPX(X_train_advanced, Y_train, X_dev_advanced, Y_dev, X_test_advanced, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.484064   0.567104  0.526324  0.627778  0.526324,
    f1_score  precision    recall  accuracy     AUROC
 0  0.488671   0.577794  0.530667     0.628  0.530667)

In [55]:
late_fuse_MLPX(X_train_advanced, Y_train, X_dev_advanced, Y_dev, X_test_advanced, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0   0.47744   0.534332  0.515588  0.612963  0.515588,
    f1_score  precision    recall  accuracy     AUROC
 0  0.505235   0.572088  0.535467     0.625  0.535467)

In [56]:
late_fuse_MLPX(X_train_advanced, Y_train, X_dev_advanced, Y_dev, X_test_advanced, Y_test)

C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

(   f1_score  precision  recall  accuracy  AUROC
 0  0.386364   0.314815     0.5   0.62963    0.5,
    f1_score  precision  recall  accuracy  AUROC
 0  0.384615     0.3125     0.5     0.625    0.5)

In [57]:
late_fuse_MLPX(X_train_advanced, Y_train, X_dev_advanced, Y_dev, X_test_advanced, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.477226   0.561081  0.522794  0.625926  0.522794,
    f1_score  precision    recall  accuracy     AUROC
 0  0.476306   0.595598  0.529333     0.633  0.529333)

In [58]:
late_fuse_MLPX(X_train_advanced, Y_train, X_dev_advanced, Y_dev, X_test_advanced, Y_test)

(   f1_score  precision  recall  accuracy  AUROC
 0  0.442958   0.532771   0.525  0.453704  0.525,
    f1_score  precision    recall  accuracy     AUROC
 0  0.454904   0.555361  0.540533     0.468  0.540533)

In [59]:
late_fuse_MLPX(X_train_advanced, Y_train, X_dev_advanced, Y_dev, X_test_advanced, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.445136   0.534507  0.526471  0.455556  0.526471,
    f1_score  precision    recall  accuracy     AUROC
 0  0.457966   0.555024  0.541067      0.47  0.541067)

In [60]:
late_fuse_MLPX(X_train_advanced, Y_train, X_dev_advanced, Y_dev, X_test_advanced, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.448707   0.548903  0.536029  0.461111  0.536029,
    f1_score  precision    recall  accuracy     AUROC
 0  0.450763    0.55425  0.538933     0.465  0.538933)

In [61]:
late_fuse_MLPX(X_train_advanced, Y_train, X_dev_advanced, Y_dev, X_test_advanced, Y_test)

C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

(   f1_score  precision  recall  accuracy  AUROC
 0  0.386364   0.314815     0.5   0.62963    0.5,
    f1_score  precision  recall  accuracy  AUROC
 0  0.384615     0.3125     0.5     0.625    0.5)

In [62]:
late_fuse_MLPX(X_train_advanced, Y_train, X_dev_advanced, Y_dev, X_test_advanced, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.480293   0.530031  0.514706  0.609259  0.514706,
    f1_score  precision    recall  accuracy     AUROC
 0  0.506468   0.570307  0.535467     0.624  0.535467)

In [82]:
run_multiple_times(X_train_advanced, Y_train, X_dev_advanced, Y_dev, X_test_advanced, Y_test)

C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

Average Development: 
   f1_score  precision   recall  accuracy    AUROC
0  0.466087    0.52597  0.51925     0.605  0.51925 
Average Test: 
   f1_score  precision    recall  accuracy     AUROC
0  0.478189   0.553158  0.530733   0.61255  0.530733


# Fine-tuned_Bert:

In [63]:
from transformers import BertTokenizer, BertForSequenceClassification
from utils import *
# Set up GPU or CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Load pre-trained tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')
model = BertForSequenceClassification.from_pretrained('bert-base-cased', num_labels=2)
model.to(device)

tokenizer, model = fine_tune(df_train_original, tokenizer, model)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

Finished epoch 1 with average training loss of 0.5718760155421451.
Finished epoch 2 with average training loss of 0.4579912252668151.
Finished epoch 3 with average training loss of 0.3781905441467923.
Finished epoch 4 with average training loss of 0.3154261342452881.
Finished epoch 5 with average training loss of 0.2740296280025539.
Finished epoch 6 with average training loss of 0.2410158093150397.
Finished epoch 7 with average training loss of 0.2187193022914847.
Finished epoch 8 with average training loss of 0.2022989973061739.
Finished epoch 9 with average training loss of 0.18523835298374183.
Finished epoch 10 with average training loss of 0.1775378146159806.


In [64]:
y_train_proba_bert = predict_proba_from_fine_tuned(df_train, tokenizer, model)
y_dev_proba_bert = predict_proba_from_fine_tuned(df_dev, tokenizer, model)
y_test_proba_bert = predict_proba_from_fine_tuned(df_test, tokenizer, model)


In [65]:
X_train_bert = fuse_proba(y_train_proba_bert, y_train_proba_ResNet)
X_dev_bert = fuse_proba(y_dev_proba_bert, y_dev_proba_ResNet)
X_test_bert = fuse_proba(y_test_proba_bert, y_test_proba_ResNet)

bert_ResNet_results = late_fuse_MLPX(X_train_bert, Y_train, X_dev_bert, Y_dev, X_test_bert, Y_test)

In [66]:
bert_ResNet_results

(   f1_score  precision    recall  accuracy     AUROC
 0  0.528213   0.528139  0.528676  0.555556  0.528676,
    f1_score  precision  recall  accuracy   AUROC
 0  0.581919   0.581565  0.5824     0.606  0.5824)

In [67]:
late_fuse_MLPX(X_train_bert, Y_train, X_dev_bert, Y_dev, X_test_bert, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.488824   0.548916  0.523088  0.618519  0.523088,
    f1_score  precision    recall  accuracy     AUROC
 0   0.51438    0.57609  0.540133    0.6265  0.540133)

In [68]:
late_fuse_MLPX(X_train_bert, Y_train, X_dev_bert, Y_dev, X_test_bert, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.535477    0.53592  0.535294   0.57037  0.535294,
    f1_score  precision  recall  accuracy   AUROC
 0  0.582797   0.583324  0.5824     0.611  0.5824)

In [69]:
late_fuse_MLPX(X_train_bert, Y_train, X_dev_bert, Y_dev, X_test_bert, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.536134   0.563018  0.545294  0.616667  0.545294,
    f1_score  precision  recall  accuracy   AUROC
 0  0.551747    0.58139  0.5592     0.626  0.5592)

In [70]:
late_fuse_MLPX(X_train_bert, Y_train, X_dev_bert, Y_dev, X_test_bert, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.529097    0.52905  0.529706  0.555556  0.529706,
    f1_score  precision    recall  accuracy     AUROC
 0  0.583191   0.582805  0.583733     0.607  0.583733)

In [71]:
late_fuse_MLPX(X_train_bert, Y_train, X_dev_bert, Y_dev, X_test_bert, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.535477    0.53592  0.535294   0.57037  0.535294,
    f1_score  precision  recall  accuracy   AUROC
 0  0.583351   0.584177  0.5828    0.6125  0.5828)

In [72]:
late_fuse_MLPX(X_train_bert, Y_train, X_dev_bert, Y_dev, X_test_bert, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.496223   0.545788  0.524265  0.614815  0.524265,
    f1_score  precision  recall  accuracy   AUROC
 0  0.521721   0.580516  0.5444    0.6285  0.5444)

In [73]:
late_fuse_MLPX(X_train_bert, Y_train, X_dev_bert, Y_dev, X_test_bert, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.514448   0.557778  0.534412  0.618519  0.534412,
    f1_score  precision  recall  accuracy   AUROC
 0  0.529595   0.583834  0.5488      0.63  0.5488)

In [74]:
late_fuse_MLPX(X_train_bert, Y_train, X_dev_bert, Y_dev, X_test_bert, Y_test)

(   f1_score  precision    recall  accuracy     AUROC
 0  0.535094   0.558883  0.543382  0.612963  0.543382,
    f1_score  precision    recall  accuracy     AUROC
 0  0.549319   0.582751  0.558267    0.6275  0.558267)

In [75]:
late_fuse_MLPX(X_train_bert, Y_train, X_dev_bert, Y_dev, X_test_bert, Y_test)

C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\Hisha\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1334: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average,

(   f1_score  precision  recall  accuracy  AUROC
 0  0.386364   0.314815     0.5   0.62963    0.5,
    f1_score  precision  recall  accuracy  AUROC
 0  0.384615     0.3125     0.5     0.625    0.5)

In [79]:
run_multiple_times(X_train_bert, Y_train, X_dev_bert, Y_dev, X_test_bert, Y_test)

Average Development: 
   f1_score  precision    recall  accuracy     AUROC
0  0.521761   0.553592  0.536015  0.607963  0.536015 
Average Test: 
   f1_score  precision    recall  accuracy     AUROC
0  0.546444   0.583236  0.558667    0.6258  0.558667
